## 5 steps of using tensorboard

* From TF graph, decide which node you want to annotate
 - with tf.name_scope("test") as scope: # Node name
 - tf.hisogram_summary("weights", W)  # 스칼라 값이 아닌 값
 - tf.scalar_summary("accuracy", accuracy) # 하나의 스칼라 값
* Merge all summaries
 - merged = tf.merge_all_summaries() # 모든 정의해놓은 summary data들을 merge해서 하나의 operation으로 연결을 시켜 줌
* Create writer
 - writer = tf.train.SummaryWriter("/tmp/mnist_logs", sess.graph_def) # 어디에다 쓸건지 정해줌
* Run summary merge and add_summary
 - summary = sess.run(merged,...); writer.add_summary(summary); # 실행, 실행결과 작성
* Launch Tensorboard
 - tensorboard --logdir=/tmp/mnist_logs # 결과 보기

In [1]:
import tensorflow as tf
import numpy as np

### Name variables

In [2]:
xy = np.loadtxt('train_data/lab9-1_train.txt', unpack=True)

x_data = np.transpose(xy[0:-1])
y_data = np.reshape(xy[-1], (4,1))

X = tf.placeholder(tf.float32, name="X-input")
Y = tf.placeholder(tf.float32, name="Y-input")

W1 = tf.Variable(tf.random_uniform( [2,2], -1.0, 1.0), name = "weight1")
W2 = tf.Variable(tf.random_uniform( [2,1], -1.0, 1.0), name = "weight2")

b1 = tf.Variable(tf.zeros([2]), name="Bias1")
b2 = tf.Variable(tf.zeros([1]), name="Bias2")

### Add cope for better graph hierarch

In [3]:
# Our hypothesis
with tf.name_scope("layer2") as scope:
    L2 = tf.sigmoid(tf.matmul(X, W1) + b1)
    
with tf.name_scope("hypohesis") as scope:
    hypothesis = tf.sigmoid(tf.matmul(L2, W2) + b2)
    
# cost function
with tf.name_scope("cost") as scope:
    cost = -tf.reduce_mean(Y*tf.log(hypothesis)+(1-Y)* tf.log(1-hypothesis))
    cost_summ = tf.summary.scalar("cost", cost)

# Minimize
a = tf.Variable(0.1) # Learning rate, alpha
with tf.name_scope("train") as scope:
    optimizer = tf.train.GradientDescentOptimizer(a)
    train = optimizer.minimize(cost)

### Add histogram

In [4]:
w1_hist = tf.summary.histogram("weights1", W1)
w2_hist = tf.summary.histogram("weights2", W2)

b1_hist = tf.summary.histogram("biases1", b1)
b2_hist = tf.summary.histogram("biases2", b2)

y_hist = tf.summary.histogram("y", Y)

In [5]:
init = tf.global_variables_initializer()

### merge summaries and crate writer after creating session & run merged summary and write (add summary)

In [6]:
# Launch the graph
with tf.Session() as sess:
    
    # tensorboard --logdir=./logs/xor_logs
    merged = tf.summary.merge_all()
    writer = tf.summary.FileWriter("./logs/xor_logs", sess.graph)
    
    sess.run(init)
    
    # fit the line.
    for step in range(20001):
        sess.run(train, feed_dict={X:x_data, Y:y_data})
        if step % 4000 == 0:
            summary, _ = sess.run([merged, train], feed_dict={X:x_data, Y:y_data})
            writer.add_summary(summary, step)
            
    # Test model
    correct_prediction = tf.equal(tf.floor(hypothesis + 0.5), Y)
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print(sess.run([hypothesis, tf.floor(hypothesis+0.5), correct_prediction, accuracy], feed_dict={X:x_data, Y:y_data}))
    print("Accuracy:", accuracy.eval({X:x_data, Y:y_data}))

[array([[ 0.00316553],
       [ 0.99534577],
       [ 0.49903515],
       [ 0.50220847]], dtype=float32), array([[ 0.],
       [ 1.],
       [ 0.],
       [ 1.]], dtype=float32), array([[ True],
       [ True],
       [False],
       [False]], dtype=bool), 0.5]
Accuracy: 0.5


### Launch tensorboard

* tensorboard --logdir=/logs/xor_logs
* (You can navigate to http://localhost.6006)

In [7]:
import tensorbord_helper as tbh

In [8]:
tbh.show_graph(tf.get_default_graph().as_graph_def())